# Automatización Programación a Base de Datos

## Importamos Librerias

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta

## Definimos funciones de extracción

In [3]:
FIRST_ROW = 12
LAST_ROW = 61

def int_to_excel_col(n):
    result = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        result = chr(65 + remainder) + result
    return result

def excel_col_to_int(col):
    result = 0
    for char in col:
        result = result * 26 + (ord(char.upper()) - ord('A') + 1)
    return result

def next_excel_col(col):
    col_int = excel_col_to_int(col)
    next_col_int = col_int + 1
    return int_to_excel_col(next_col_int)

def extraer_planificacion(file, sheet):
    df_planificacion = pd.read_excel(file, sheet_name=sheet, header=None)
    df_planificacion.index = range(1, len(df_planificacion) + 1)
    df_planificacion.columns = [int_to_excel_col(i) for i in range(1, len(df_planificacion.columns) + 1)]

    return df_planificacion

def extraer_bts(file, sheet):
    df_bts = pd.read_excel(file, sheet_name=sheet, header=0)
    df_bts.index = range(1, len(df_bts) + 1)
    df_bts = df_bts[["N° Referencia", "Nombre programa", "Nombre del BT", "Abrev."]]
    df_bts.drop_duplicates(subset=["Abrev."], keep="first", inplace=True)
    
    return df_bts

def extraer_descargas(df_planificacion):
    df_descargas = pd.DataFrame({"Fecha": [], "Abrev.": [], "Volumen": [], "Columna": []})
    columnas = ["M", "S", "Y", "AE", "AK", "AQ", "AW", "BC", "BI", "BO", "BU", "CA", "CF", "CK", "CP", "CU", "DA"]
    for col in columnas:
        descargas_parciales = df_planificacion.loc[FIRST_ROW:LAST_ROW, ["B", col, next_excel_col(col)]].dropna()
        descargas_parciales.columns = ["Fecha", "Abrev.", "Volumen"]
        descargas_parciales["Columna"] = [col] * len(descargas_parciales)
        df_descargas = pd.concat([df_descargas, descargas_parciales], ignore_index=True)
    df_descargas = df_descargas[(df_descargas["Abrev."] != "Enap") & (df_descargas["Abrev."] != "Puma")]

    return df_descargas

def extraer_distancias(file, sheet):
    df_distancias = pd.read_excel(file, sheet_name=sheet, header=3, index_col=0)
    df_distancias.dropna(axis=1, how='all', inplace=True)
    df_distancias.dropna(axis=0, how='all', inplace=True)
    return df_distancias

def extraer_programas(df_planificacion):
    df_programas = df_planificacion.loc[FIRST_ROW:LAST_ROW, ["B", "J"]].dropna()
    df_programas.columns = ["Fecha", "Nombre programa"]
    return df_programas

## Extracción de Información

In [4]:
PATH_PROGRAMACION = "Programacion Descarga Importaciones 22 de Sep.xlsx"
PATH_DISTANCIAS = "Distancias entre puertos.xlsx"

### Paso 1:

Extraemos todos los programas, con sus respectivos buques y abreviaturas

In [5]:
df_bts = extraer_bts(PATH_PROGRAMACION, "Buques")
df_bts

,N° Referencia,Nombre programa,Nombre del BT,Abrev.
1,CC 46/25,46/25 Sakura Voyager,Sakura Voyager,SV
2,CC 117/25,117/25 Yaca,Yaca,Yac
3,CC 135/25,135/25 Seaways Kenosha,Seaways Kenosha,SK
4,CC 19/25,19/25 Cururo,Cururo,Cur
5,CC 47/25,47/25 San Jack,San Jack,SJ
6,CC 102/25,102/25 Pintail Pacific,Pintail Pacific,PP
9,CC 75/25,75/25 Flora Express,Flora Express,FE
10,CC 21/25,21/25 Seaways Athens,Seaways Athens,SA
11,CC 48/25,48/25 STI Mighty,STI Mighty,SM
12,CC 90/25,90/25 Dat Mercury,Dat Mercury,DM


### Paso 2: 

Creamos un dataframe con los productos y las plantas presentes en la planificación.

In [6]:
df_productos = pd.DataFrame({"Producto": ["Diesel A1", "Gas 93", "Gas 97", "Jet A1"]})
df_plantas = pd.DataFrame({"Planta": ["PLANTA IQUIQUE", "PLANTA MEJILLONES", "PLANTA CALDERA",
                                      "TERMINAL TPI", "OXIQUIM QUINTERO", "OXIQUIM CORONEL",
                                      "PLANTA PUREO", "ENAP QUINTERO", "BT PUMA"],
                           "Ciudad": ["Iquique", "Mejillones", "Caldera", "Quintero", "Quintero",
                                      "Coronel", "Calbuco", np.nan, np.nan],
                           "Alias": ["Iquique", "Mejillones", "Caldera", "TPI", "Oxiquim Quintero",
                                      "Coronel", "Pureo", "Quintero", "Puma"]})

### Paso 3: 

Producto cruz entre plantas y productos para generar todas las combinaciones posibles.

In [7]:
df_productos_plantas = df_productos.merge(df_plantas, how='cross')

### Paso 4: 

Asignamos a cada par producto-planta la columna correspondiente en la planificación.

In [8]:
df_productos_plantas["Columna"] = ["M", "S", "Y", "AE", "BC", "BI", "BO", "BU", "CA",
                                   np.nan, "DA", np.nan, "AQ", np.nan, np.nan, np.nan, np.nan, "CK",
                                   np.nan, np.nan, np.nan, np.nan, "AW", np.nan, np.nan, np.nan, "CP",
                                   np.nan, "CU", np.nan, "AK", np.nan, np.nan, np.nan, np.nan, "CF"]

### Paso 5: 

Limpiamos el dataframe eliminando las filas sin columna asignada.

In [9]:
df_productos_plantas.dropna(subset=["Columna"], inplace=True)
df_productos_plantas

,Producto,Planta,Ciudad,Alias,Columna
0,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,M
1,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,S
2,Diesel A1,PLANTA CALDERA,Caldera,Caldera,Y
3,Diesel A1,TERMINAL TPI,Quintero,TPI,AE
4,Diesel A1,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,BC
5,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,BI
6,Diesel A1,PLANTA PUREO,Calbuco,Pureo,BO
7,Diesel A1,ENAP QUINTERO,NaN,Quintero,BU
8,Diesel A1,BT PUMA,NaN,Puma,CA
10,Gas 93,PLANTA MEJILLONES,Mejillones,Mejillones,DA


### Paso 6:

Extraemos la planificación y sus respectivas descargas. Se ignoran descargas realizadas por el Puma y Enap.

In [10]:
df_planificacion = extraer_planificacion(PATH_PROGRAMACION, "Planificación")
df_descargas = extraer_descargas(df_planificacion)
df_descargas

c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Fecha,Abrev.,Volumen,Columna
0,2025-09-25 00:00:00,FE,4000,M
1,2025-09-26 00:00:00,FE,4000,M
2,2025-09-30 00:00:00,SM,6000,M
3,2025-10-01 00:00:00,SM,6000,M
5,2025-10-14 00:00:00,PK,6000,M
...,...,...,...,...
120,2025-11-01 00:00:00,Caldera,-6000,CA
121,2025-09-23 00:00:00,PP,15260,CK
122,2025-10-06 00:00:00,TPI,-13000,CK
123,2025-10-10 00:00:00,TPI,-9245,CK


### Paso 7:

Mediante la el atributo "Columna" asociamos cada descarga a un par producto-planta.

In [11]:
df_descargas_productos_plantas = df_descargas.merge(df_productos_plantas, on=["Columna"]).drop(columns=["Columna"])
df_descargas_productos_plantas

,Fecha,Abrev.,Volumen,Producto,Planta,Ciudad,Alias
0,2025-09-25 00:00:00,FE,4000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique
1,2025-09-26 00:00:00,FE,4000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique
2,2025-09-30 00:00:00,SM,6000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique
3,2025-10-01 00:00:00,SM,6000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique
4,2025-10-14 00:00:00,PK,6000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique
...,...,...,...,...,...,...,...
87,2025-11-01 00:00:00,Caldera,-6000,Diesel A1,BT PUMA,NaN,Puma
88,2025-09-23 00:00:00,PP,15260,Gas 93,BT PUMA,NaN,Puma
89,2025-10-06 00:00:00,TPI,-13000,Gas 93,BT PUMA,NaN,Puma
90,2025-10-10 00:00:00,TPI,-9245,Gas 93,BT PUMA,NaN,Puma


### Paso 8:

Mediante la abreviatura asociamos cada descarga a un programa (Por esto es importante que la abreviatura sea única, o en su defecto utilizar otro atributo de carater único como correlativo o nombre de programa).
Se eliminarán todas las descargas que no tengan la abreviatura de un buque asociada (Ej: Caldera o TPI hacia el Puma).

In [12]:
df_descargas_completo = df_descargas_productos_plantas.merge(df_bts, on=["Abrev."]).drop(columns=["Abrev."])
df_descargas_completo

,Fecha,Volumen,Producto,Planta,Ciudad,Alias,N° Referencia,Nombre programa,Nombre del BT
0,2025-09-25 00:00:00,4000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,CC 75/25,75/25 Flora Express,Flora Express
1,2025-09-26 00:00:00,4000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,CC 75/25,75/25 Flora Express,Flora Express
2,2025-09-30 00:00:00,6000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,CC 48/25,48/25 STI Mighty,STI Mighty
3,2025-10-01 00:00:00,6000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,CC 48/25,48/25 STI Mighty,STI Mighty
4,2025-10-14 00:00:00,6000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,CC 50/25,50/25 Pacific Kohinoor,Pacific Kohinoor
...,...,...,...,...,...,...,...,...,...
70,2025-10-14 00:00:00,9000,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,CC 118/25,118/25 Andean Sun,Andean Sun
71,2025-10-15 00:00:00,9000,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,CC 118/25,118/25 Andean Sun,Andean Sun
72,2025-10-27 00:00:00,7500,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,CC 130/25,130/25 Seaways Stamford,Seaways Stamford
73,2025-10-28 00:00:00,7500,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,CC 130/25,130/25 Seaways Stamford,Seaways Stamford


### Paso 9:

Agrupamos todas las descargas de un programa que son en días consecutivos de un mismo producto y en una misma planta como una sola descarga.

In [13]:
df_descargas_completo = df_descargas_completo[["Fecha", "N° Referencia", "Nombre programa", "Nombre del BT",
                                               "Producto", "Planta", "Ciudad", "Alias", "Volumen"]]

df_descargas_completo["Fecha"] = pd.to_datetime(df_descargas_completo["Fecha"])
df_descargas_completo["diff"] = (
    df_descargas_completo.groupby(["Nombre programa", "Producto", "Planta"])["Fecha"]
    .diff()
    .dt.days
)

df_descargas_completo["Grupo"] = (df_descargas_completo["diff"] != 1).cumsum()
df_descargas_agrupadas = (
    df_descargas_completo.groupby("Grupo", as_index=False).agg({
        "Fecha": ["min", "max"],   # rango de fechas
        "Volumen": "sum"
    })
)
df_descargas_agrupadas.columns = ["Grupo", "Fecha inicio", "Fecha fin", "Volumen total"]

df_descargas_completo_sin_duplicados = df_descargas_completo.drop_duplicates(subset=["Grupo"])
df_descargas_agrupadas = df_descargas_agrupadas.merge(
    df_descargas_completo_sin_duplicados, on="Grupo").drop(columns=["diff", "Fecha", "Volumen"])
df_descargas_agrupadas = df_descargas_agrupadas[[
    "Nombre programa", "N° Referencia", "Nombre del BT", "Producto", "Planta", "Ciudad", "Alias",
    "Fecha inicio", "Fecha fin", "Volumen total"]]

# Ordenar por Nombre programa y Fecha inicio, y agregar N° Descarga
df_descargas_agrupadas = df_descargas_agrupadas.sort_values(by=["Nombre programa", "Fecha inicio"])
df_descargas_agrupadas["N° Descarga"] = (
    df_descargas_agrupadas.groupby("Nombre programa").cumcount() + 1
)
df_descargas_agrupadas

,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,N° Descarga
37,102/25 Pintail Pacific,CC 102/25,Pintail Pacific,Gas 93,BT PUMA,NaN,Puma,2025-09-23,2025-09-23,15260,1
30,102/25 Pintail Pacific,CC 102/25,Pintail Pacific,Gas 93,TERMINAL TPI,Quintero,TPI,2025-09-25,2025-09-25,34000,2
32,103/25 Seaways Wheat /High Challenge,CC 103/25,Seaways Wheat /High Challenge,Gas 97,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,2025-10-23,2025-10-24,8000,1
31,103/25 Seaways Wheat /High Challenge,CC 103/25,Seaways Wheat /High Challenge,Gas 93,TERMINAL TPI,Quintero,TPI,2025-10-28,2025-10-29,40000,2
8,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-09,2025-10-10,30000,1
35,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-14,2025-10-15,18000,2
3,119/25 Seaways Dwarka,CC 119/25,Seaways Dwarka,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-10-23,2025-10-24,12000,1
12,119/25 Seaways Dwarka,CC 119/25,Seaways Dwarka,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-28,2025-10-29,36000,2
25,130/25 Seaways Stamford,CC 130/25,Seaways Stamford,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-23,2025-10-24,32000,1
36,130/25 Seaways Stamford,CC 130/25,Seaways Stamford,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-27,2025-10-28,15000,2


### Paso 10:

Extraemos todos los programas de la planificación.

In [14]:
df_programas = extraer_programas(df_planificacion)
df_programas.columns = ["ETA", "Nombre programa"]
df_programas

,ETA,Nombre programa
16,2025-09-25 00:00:00,75/25 Flora Express
20,2025-09-29 00:00:00,48/25 STI Mighty
22,2025-10-01 00:00:00,21/25 Seaways Athens
27,2025-10-06 00:00:00,118/25 Andean Sun
28,2025-10-07 00:00:00,49/25 Pacific Sentinel
31,2025-10-10 00:00:00,90/25 Dat Mercury
34,2025-10-13 00:00:00,50/25 Pacific Kohinoor
35,2025-10-14 00:00:00,20/25 Seaways Kolberg
36,2025-10-15 00:00:00,74/25 Gem Sapphire
37,2025-10-16 00:00:00,130/25 Seaways Stamford


### Paso 10:

Asociamos a cada programa sus respectivas descargas, agregamos la ETA a la primera descarga de cada programa.

* Nota Importante: Esto ignorará todas las descargas asociadas a programas cuya ETA estimada sea anterior al inicio de la programación.

In [15]:
df_descargas_agrupadas = df_programas.merge(df_descargas_agrupadas, on="Nombre programa")
df_descargas_agrupadas["ETA"] = df_descargas_agrupadas["ETA"][[True if descarga == 1 else False for descarga in df_descargas_agrupadas["N° Descarga"]]]
df_descargas_agrupadas

,ETA,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,N° Descarga
0,2025-09-25 00:00:00,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-25,2025-09-26,8000,1
1,NaN,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-03,2025-10-04,42398,2
2,2025-09-29 00:00:00,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-30,2025-10-01,12000,1
3,NaN,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-04,2025-10-05,36000,2
4,2025-10-01 00:00:00,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,2025-10-04,2025-10-05,15000,1
5,NaN,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-07,2025-10-08,33000,2
6,2025-10-06 00:00:00,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-09,2025-10-10,30000,1
7,NaN,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-14,2025-10-15,18000,2
8,2025-10-07 00:00:00,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-10-09,2025-10-10,12000,1
9,NaN,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-14,2025-10-15,36000,2


### Paso 11:

Extraer matriz de distancias y pasar a tiempos aproximando en horas al entero superior

In [16]:
matriz_de_distancias = extraer_distancias(PATH_DISTANCIAS, "Datos")
VELOCIDAD_MEDIA = 12  # nudos
# Calcular matriz de tiempos aproximando en horas al entero superior
matriz_de_tiempos = np.ceil(matriz_de_distancias / VELOCIDAD_MEDIA).astype(int)
matriz_de_tiempos


,ARICA,IQUIQUE,TOCOPILLA,MEJILLONES,ANTOFAGASTA,BARQUITO,CALDERA,HUASCO,GUAYACAN,QUINTERO,SAN VICENTE,CORONEL,PUERTO MONTT,CALBUCO,CHACABUCO,PUNTA ARENAS,CABO NEGRO
Arica,0,9,19,24,28,40,44,51,59,73,93,95,124,122,141,192,193
Iquique,9,0,10,15,19,32,35,43,51,64,85,87,116,113,133,183,185
Tocopilla,19,10,0,6,10,23,26,33,41,55,75,77,107,104,123,174,175
Mejillones,24,15,6,0,6,19,22,30,38,51,72,74,102,100,119,170,172
Antofagasta,28,19,10,6,0,15,18,26,34,48,68,70,99,97,116,166,168
Barquito,40,32,23,19,15,0,5,12,20,33,54,56,86,83,102,153,154
Caldera,44,35,26,22,18,5,0,8,17,31,51,53,82,80,99,149,151
Huasco,51,43,33,30,26,12,8,0,9,23,44,46,75,72,92,142,143
Guayacan,59,51,41,38,34,20,17,9,0,16,36,38,67,65,84,135,136
Quintero,73,64,55,51,48,33,31,23,16,0,22,24,54,51,70,121,122


### Paso 12:

Calcular ETAs según matriz de tiempos

In [17]:
def rellenar_etas(df_descargas, df_tiempos):
    # Copiamos para no modificar el original
    df = df_descargas.copy()
    
    for programa, group in df.groupby('Nombre programa'):
        group = group.sort_values('N° Descarga').copy()
        for i in range(len(group)):
            if pd.isna(group.iloc[i]['ETA']):
                if i == 0:
                    continue  # No hay anterior
                prev = group.iloc[i-1]
                ciudad_origen = prev['Ciudad']
                ciudad_destino = group.iloc[i]['Ciudad']

                # Hora de salida: fecha fin anterior a las 23:00
                hora_salida = prev['Fecha fin'].replace(hour=23, minute=0, second=0)
                
                if pd.isna(ciudad_origen) or pd.isna(ciudad_destino):
                    df.loc[group.index[i], 'ETA'] = hora_salida
                    continue
                
                # Horas de viaje desde la matriz
                horas_viaje = df_tiempos.loc[ciudad_origen, ciudad_destino.upper()]
                horas_viaje = horas_viaje
                
                # Rellenar ETA
                df.loc[group.index[i], 'ETA'] = hora_salida + timedelta(hours=int(horas_viaje))

    return df

In [18]:
df_descargas_agrupadas = rellenar_etas(df_descargas_agrupadas, matriz_de_tiempos)
df_descargas_agrupadas

,ETA,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,N° Descarga
0,2025-09-25 00:00:00,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-25,2025-09-26,8000,1
1,2025-09-29 15:00:00,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-03,2025-10-04,42398,2
2,2025-09-29 00:00:00,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-30,2025-10-01,12000,1
3,2025-10-02 14:00:00,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-04,2025-10-05,36000,2
4,2025-10-01 00:00:00,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,2025-10-04,2025-10-05,15000,1
5,2025-10-05 23:00:00,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-07,2025-10-08,33000,2
6,2025-10-06 00:00:00,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-09,2025-10-10,30000,1
7,2025-10-14 01:00:00,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-14,2025-10-15,18000,2
8,2025-10-07 00:00:00,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-10-09,2025-10-10,12000,1
9,2025-10-11 21:00:00,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-14,2025-10-15,36000,2


### Paso 13:

Ajustamos al formato de la Base de Datos.

In [19]:
df_BD = df_descargas_agrupadas[["N° Referencia", "Nombre del BT", "Producto", "Alias", "Volumen total", "ETA", "Fecha fin"]]
df_BD.columns = ["CC", "Nombre BT", "Producto", "Puerto", "Volumen", "ETA", "Fin descarga"]
df_BD

,CC,Nombre BT,Producto,Puerto,Volumen,ETA,Fin descarga
0,CC 75/25,Flora Express,Diesel A1,Iquique,8000,2025-09-25 00:00:00,2025-09-26
1,CC 75/25,Flora Express,Diesel A1,TPI,42398,2025-09-29 15:00:00,2025-10-04
2,CC 48/25,STI Mighty,Diesel A1,Iquique,12000,2025-09-29 00:00:00,2025-10-01
3,CC 48/25,STI Mighty,Diesel A1,Mejillones,36000,2025-10-02 14:00:00,2025-10-05
4,CC 21/25,Seaways Athens,Diesel A1,Oxiquim Quintero,15000,2025-10-01 00:00:00,2025-10-05
5,CC 21/25,Seaways Athens,Diesel A1,TPI,33000,2025-10-05 23:00:00,2025-10-08
6,CC 118/25,Andean Sun,Diesel A1,Mejillones,30000,2025-10-06 00:00:00,2025-10-10
7,CC 118/25,Andean Sun,Diesel A1,Coronel,18000,2025-10-14 01:00:00,2025-10-15
8,CC 49/25,Pacific Sentinel,Diesel A1,Caldera,12000,2025-10-07 00:00:00,2025-10-10
9,CC 49/25,Pacific Sentinel,Diesel A1,Mejillones,36000,2025-10-11 21:00:00,2025-10-15
